In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     


── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


[1] "cyclistic-nyc"

# Cyclistic Bike-Share Analysis — Google BI Certificate Final Project

This business intelligence (BI) project analyzes New York City's Cyclistic bike-share data as part of the Google Business Intelligence Certificate.

---

## 📅 Project Overview
This project focuses on uncovering key insights from NYC bike-share usage in 2019 and 2020. It uses Tableau for visualization and public datasets including trip data, weather information, and zip code boundaries.

The analysis is divided into three main dashboard tabs:

1. **Summer Trends** — Seasonal trip behavior in July, August, September
2. **Seasonality** — Monthly trip totals and neighborhood usage heatmaps
3. **Top Trips** — Neighborhoods with the highest total ride durations

---

## 🔍 Executive Summary
**Business Needs:** Cyclistic aims to understand usage behavior to optimize marketing and infrastructure strategies.

**Goals:**
- Identify peak trip months and popular neighborhoods
- Understand differences between casual riders and subscribers
- Inform expansion efforts and targeted outreach

**BI Methods:**
- SQL queries in BigQuery to generate target tables
- Calculated fields and filters in Tableau for analysis
- Dashboards designed with filters, parameter controls, and drill-down actions

**Key Insights:**
- Subscribers consistently make more trips, especially in warmer months
- Lower East Side, Chelsea, and Clinton areas have highest ride durations
- Summer months show peaks in ride volume and longer trip durations

---

## 🎨 Dashboard Link
> 🔗 https://public.tableau.com/app/profile/saud.alshammari/viz/CyclisticBike-ShareAnalysisGoogleBICertificateFinalProject_17465159433040/SummerTrends

---

## 📂 Files and Resources
- Executive Summary (PDF)
- Tableau Workbook (Public link)
- Source CSV Files:
  - `Examplar Cyclistic summer time.csv`
  - `Examplar Cyclistic year time.csv`

## 🧠 SQL Code: Target Tables

### 📌 Target Table 1: Full-Year Trip Summary
```sql
SELECT
  TRI.usertype,
  ZIPSTART.zip_code AS zip_code_start,
  ZIPSTARTNAME.borough AS borough_start,
  ZIPSTARTNAME.neighborhood AS neighborhood_start,
  ZIPEND.zip_code AS zip_code_end,
  ZIPENDNAME.borough AS borough_end,
  ZIPENDNAME.neighborhood AS neighborhood_end,
  DATE_ADD(DATE(TRI.starttime), INTERVAL 5 YEAR) AS start_day,
  DATE_ADD(DATE(TRI.stoptime), INTERVAL 5 YEAR) AS stop_day,
  WEA.temp AS day_mean_temperature,
  WEA.wdsp AS day_mean_wind_speed,
  WEA.prcp AS day_total_precipitation,
  ROUND(CAST(TRI.tripduration / 60 AS INT64), -1) AS trip_minutes,
  COUNT(TRI.bikeid) AS trip_count
FROM `bigquery-public-data.new_york_citibike.citibike_trips` AS TRI
INNER JOIN `bigquery-public-data.geo_us_boundaries.zip_codes` ZIPSTART
  ON ST_WITHIN(ST_GEOGPOINT(TRI.start_station_longitude, TRI.start_station_latitude), ZIPSTART.zip_code_geom)
INNER JOIN `bigquery-public-data.geo_us_boundaries.zip_codes` ZIPEND
  ON ST_WITHIN(ST_GEOGPOINT(TRI.end_station_longitude, TRI.end_station_latitude), ZIPEND.zip_code_geom)
INNER JOIN `bigquery-public-data.noaa_gsod.gsod20*` AS WEA
  ON PARSE_DATE("%Y%m%d", CONCAT(WEA.year, WEA.mo, WEA.da)) = DATE(TRI.starttime)
INNER JOIN `legalbi.sandbox.zipcodes` AS ZIPSTARTNAME
  ON ZIPSTART.zip_code = CAST(ZIPSTARTNAME.zip AS STRING)
INNER JOIN `legalbi.sandbox.zipcodes` AS ZIPENDNAME
  ON ZIPEND.zip_code = CAST(ZIPENDNAME.zip AS STRING)
WHERE
  WEA.wban = '94728'
  AND EXTRACT(YEAR FROM DATE(TRI.starttime)) BETWEEN 2014 AND 2015
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13
```

### 📌 Target Table 2: Summer Season Trip Summary
```sql
SELECT
  TRI.usertype,
  TRI.start_station_longitude,
  TRI.start_station_latitude,
  TRI.end_station_longitude,
  TRI.end_station_latitude,
  ZIPSTART.zip_code AS zip_code_start,
  ZIPSTARTNAME.borough AS borough_start,
  ZIPSTARTNAME.neighborhood AS neighborhood_start,
  ZIPEND.zip_code AS zip_code_end,
  ZIPENDNAME.borough AS borough_end,
  ZIPENDNAME.neighborhood AS neighborhood_end,
  DATE_ADD(DATE(TRI.starttime), INTERVAL 5 YEAR) AS start_day,
  DATE_ADD(DATE(TRI.stoptime), INTERVAL 5 YEAR) AS stop_day,
  WEA.temp AS day_mean_temperature,
  WEA.wdsp AS day_mean_wind_speed,
  WEA.prcp AS day_total_precipitation,
  ROUND(CAST(TRI.tripduration / 60 AS INT64), -1) AS trip_minutes,
  TRI.bikeid
FROM `bigquery-public-data.new_york_citibike.citibike_trips` AS TRI
INNER JOIN `bigquery-public-data.geo_us_boundaries.zip_codes` ZIPSTART
  ON ST_WITHIN(ST_GEOGPOINT(TRI.start_station_longitude, TRI.start_station_latitude), ZIPSTART.zip_code_geom)
INNER JOIN `bigquery-public-data.geo_us_boundaries.zip_codes` ZIPEND
  ON ST_WITHIN(ST_GEOGPOINT(TRI.end_station_longitude, TRI.end_station_latitude), ZIPEND.zip_code_geom)
INNER JOIN `bigquery-public-data.noaa_gsod.gsod20*` AS WEA
  ON PARSE_DATE("%Y%m%d", CONCAT(WEA.year, WEA.mo, WEA.da)) = DATE(TRI.starttime)
INNER JOIN `legalbi.sandbox.zipcodes` AS ZIPSTARTNAME
  ON ZIPSTART.zip_code = CAST(ZIPSTARTNAME.zip AS STRING)
INNER JOIN `legalbi.sandbox.zipcodes` AS ZIPENDNAME
  ON ZIPEND.zip_code = CAST(ZIPENDNAME.zip AS STRING)
WHERE
  WEA.wban = '94728'
  AND DATE(TRI.starttime) BETWEEN DATE('2015-07-01') AND DATE('2015-09-30')
```

---

---

## 🤝 Acknowledgments
This project was created for the **Google Business Intelligence Certificate** on Coursera.

Tags: `#business-intelligence`, `#tableau`, `#cyclistic`, `#googlebi`, `#nyc`, `#dataviz`

---

> Contact me or view more of my work on my Tableau Public profile: https://public.tableau.com/app/profile/saud.alshammari/vizzes